<a href="https://colab.research.google.com/github/Chaaronn/Auto-PGN-Generator/blob/master/Auto_PGN_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatically generate a PGN based off given configuration.

This notebook takes user configuration and generates PGN dependant on these. I made this specifically to create new studies on Listudy, will lines that other people have played for more 'realistic' training.

It takes on average ~10 minutes to produce results, but is dependant on depth and number of lines. If you have a good PC, you can connect to a local runtime and it will produce much faster.

Github repo: [Here](https://github.com/Chaaronn/Auto-PGN-Generator)

Example Lichess study: [Here](https://lichess.org/study/IDf2Qi7W)

Example Listudy: [Here](https://listudy.org/en/studies/1y9j6m-generated-openings)

---



**Quick use:**

File > Save copy in Drive > Open this version - this allows you to run all cells from one click

Runtime > Run All > PGN will be displayed at the bottom.

Once the notebook has been ran once, you only need to run the cells below *configuration* (setup, create lines, display pgn)


**Configuration:** 

Scroll down until you see the section, edit values in the form.

Once value are edited, follow Quick Use.


---

In [1]:
#@title Get the files
%%capture

!pip install python-chess
!git clone https://github.com/Chaaronn/Auto-PGN-Generator.git
!wget https://stockfishchess.org/files/stockfish_14_linux_x64_popcnt.zip && \
unzip stockfish_14_linux_x64_popcnt.zip stockfish_14_linux_x64_popcnt/stockfish_14_x64_popcnt

In [2]:
#@title Code Imports
%%capture
import requests, random, json, os, sys, ast
import chess
import chess.pgn, chess.engine
import configparser
import datetime
from IPython.display import HTML, display
import time


In [3]:
#@title Code setup

config = configparser.ConfigParser()
try:
    config.read('Auto-PGN-Generator/setup.ini')
except IOError as e:
    print("Error reading setup.ini:", str(e))
    sys.exit(1)

stockfish_path = ("/content/stockfish_14_linux_x64_popcnt/stockfish_14_x64_popcnt")


f = open('Auto-PGN-Generator/openings.json')
try:
    opening_json = json.load(f)
    f.close()
except json.JSONDecodeError as e:
    print("Error in JSON handling"), str(e)
    f.close()
    sys.exit(1)


def get_database_from_fen(fen):
    # the range of ratings the database moves will come from
    rating_range = ast.literal_eval(config['DATABASE']['rating_range'])
    # the number of moves to return (helpful to keep this +5 from max of database_choices)
    moves_to_display = config['DATABASE']['moves_to_display']
    try:
        resp = requests.get('https://explorer.lichess.ovh/lichess', 
                            params={'variant' : 'standard', 'fen': fen, 'moves': moves_to_display, 'rating' : rating_range})
        resp.raise_for_status()  # Check for any HTTP errors
        return resp.json()
    except requests.exceptions.RequestException as e:
        print("Error making API request:", str(e))
        sys.exit(1)
     
def get_top_move(db,move_level,engine):
    # try to get a move from db, otherwise use stockfish
    try:
        move = db['moves'][move_level]['uci']
    # except is either no db moves, or not one at move_level
    except:
        move = get_stockfish_move(board, engine)
    return move

def get_stockfish_analysis(board, engine):
    # analyse given board and return Centipawns from White perspective
    # used to stop db choices being wild
    # could change to utilising Lichess online eval to save processing
    info = engine.analyse(board, chess.engine.Limit(depth=depth))
    return info['score']

def get_stockfish_move(board, engine):
    # gets top move from stockfish on given board state
    # get info from analysis    
    info = engine.analyse(board, chess.engine.Limit(depth=depth))
    # get only the top move
    move = str(info['pv'][0])
    return move

def play_opening(board,opening):
    
    # plays the opening as specifed in INI file
    try:
        opening_moves = opening_json[main_opening][0][opening]
    except KeyError as e:
        print('Error: Invalid Opening', str(e))
        sys.exit(1)
    for move in opening_moves:
        board.push_uci(move)
    return board

def clean_analysis(string):
    # fixed mate issue with this
    mate_string = 'vScore(Mate('
    if mate_string in string:
        if side.lower() == 'white':
            string = 999
            return string
        else:
            string = -999
            return string 
    # always starts the same 
    start_index = string.find('Cp(') + 3
    end_index = string.find(')', start_index)
    number_string = string[start_index:end_index]
    if number_string.startswith('+'):
        number_string = number_string[1:]  # Remove the leading '+'
    string = int(number_string)
    return string


def make_moves(board,engine,move_list,max_cp,side):
    # board.turn will return True if it whites turn, False if black
    if side.lower() == 'white':
        if board.turn:
            move = get_stockfish_move(board,engine)
            board.push_uci(move)            
        else:
            # db logic here
            move_level = random.choice(database_choices)
            move = get_top_move(move_list,move_level,engine)
            board.push_uci(move)

            # now it analyses db moves and if cp is greater than 150, gets stockfish move
            current_centipawns = str(get_stockfish_analysis(board,engine))
            # necassary to remove none int values from string
            current_centipawns = clean_analysis(current_centipawns)
            # compare cp against max
            if current_centipawns >= max_cp:
                # if move is bad, return to previous state and push sf move
                board.pop()
                move = get_stockfish_move(board,engine)
                board.push_uci(move)
            else:
                # move on, the db move was good enough
                pass
    elif side.lower() == 'black':

        if board.turns:
               # db logic here
            move_level = random.choice(database_choices)
            move = get_top_move(move_list,move_level,engine)
            board.push_uci(move)

            # now it analyses db moves and if cp is greater than 150, gets stockfish move
            current_centipawns = str(get_stockfish_analysis(board,engine))
            # necassary to remove none int values from string
            #current_centipawns = int(re.sub("[^0-9]", "", current_centipawns)) 
            current_centipawns = clean_analysis(current_centipawns)
            # compare cp against max
            if current_centipawns >= max_cp:
                # if move is bad, return to previous state and push sf move
                board.pop()
                move = get_stockfish_move(board,engine)
                board.push_uci(move)
            else:
                # move on, the db move was good enough
                pass
        else:
            move = get_stockfish_move(board,engine)
            board.push_uci(move)
    else:
        print('Error in choosing side. Ensure only White or Black is Selected')

def progress(value, max):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

# Configuration


Opening set-up must be the exact spelling as in the available openings. Otherwise, you will get an error.

You can find the openings [here.](https://raw.githubusercontent.com/Chaaronn/Auto-PGN-Generator/master/openings.json) They are represented as UCI notation.


```
# "Caro Khan" : [
        {
        "Base" : ["e2e4", "c7c6", "d2d4", "d7d5"],
        "Advance" : ["e2e4", "c7c6", "d2d4", "d7d5","e4e5"],
        "Exchange" : ["e2e4", "c7c6", "d2d4", "d7d5","e4d5", "c6d5"],
        "Main Line" : ["e2e4", "c7c6", "d2d4", "d7d5","b1c3", "d5e4", "c3d4"],
        "Modern" : ["e2e4", "c7c6", "d2d4", "d7d5", "b1d2"]
        }
```

For explainations I will use the Caro Khan from this file.

**Explainations:**
*   *main_opening* - e.g. Caro Khan
*   *variation_name* - e.g. Advance, Exchange
*    *max_variations* - this is the number of variations attempted. currently can give duplicates, so put it higher than you want. Must be given as a whole number. e.g. 10
* *moves_per_line* - the number of moves for any variations. Must be given as a whole number. e.g. 10
* *max_centipawns* - this is the maximum CP value of database moves before stockfish is reverted to. Must be given as a *positive* whole number. e.g. 250
* *threads* - the number of threads the CPU will use (more suited to local running, recommended 2)


The higher the number of variations/moves per line/depth, the longer its going to take to produce results. At depth=20, lines=10, moves=10, it takes around 15 minutes. Depth 25 around 24min. Working on reducing it.

Lower max_centipawns will lead to higher level games, so adjust this accordingly to the rating range.


In [4]:
#@title SETUP

side = "White"                          #@param["White", "Black"]
main_opening = "Kings Gambit Accepted"  #@param {type:"string"}
variation_name = "Bishops Gambit Qh4"      #@param {type:"string"}

# set-up the board
board = chess.Board()
# play out the speicifed opening. this is required or the PGN will be incorrect
board = play_opening(board,variation_name)
# set-up game to add lines later
game = chess.pgn.Game()
# add headers so on import it looks nice on Lichess
game.headers['Event'] = main_opening + ' - ' + variation_name

max_variations = 10                     #@param {type:"integer"}
current_variations = 0
# max number of moves per line
moves_per_line = 10                     #@param {type:"integer"}
# how low down the top moves we look when choosing black's moves
# load with ast otherwise its just a string
database_choices = ast.literal_eval(config['DATABASE']['database_choices'])
# max centipawn value before we use stockfish instead of db
max_centipawns = 300                    #@param {type:"integer"}
# load engine
engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)
depth = 20                              #@param {type:"integer"}
threads = 2
engine.configure({'Threads': threads}) 
print('Engine Loaded')

Engine Loaded


In [5]:
#@title Create lines
%%capture
out = display(progress(current_variations, max_variations), display_id=True)
while current_variations != max_variations:
    # start new line
    while board.fullmove_number <= moves_per_line:
        # this will have to be repeated after every move
        move_list = get_database_from_fen(board.fen())
        # make a move
        make_moves(board, engine, move_list, max_centipawns,side)

    # once we reach 10 moves, add line to pgn, reset to opening
    # added so lines always end with a final White move
    if board.turn:
            move = get_stockfish_move(board,engine)
            board.push_uci(move)
    
    game.add_line(board.move_stack)
    current_variations += 1
    board.reset()
    board = play_opening(board,variation_name)
    out.update(progress(current_variations, max_variations))

engine.quit()

In [7]:
#@title Display PGN
print(game)

[Event "Kings Gambit Accepted - Bishops Gambit Qh4"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "?"]
[Black "?"]
[Result "*"]

1. e4 ( 1. e4 e5 2. f4 exf4 3. Bc4 Qh4+ 4. Kf1 d6 5. Nc3 c6 6. d4 g5 7. Nf3 Qg4 8. h4 gxh4 9. Rxh4 Qg7 10. Rxf4 Nh6 11. e5 ) ( 1. e4 e5 2. f4 exf4 3. Bc4 Qh4+ 4. Kf1 Nc6 5. Nf3 Qe7 6. d4 d6 7. Nc3 Nf6 8. Bxf4 Bd7 9. Kf2 O-O-O 10. Re1 Bg4 11. Kg1 ) ( 1. e4 e5 2. f4 exf4 3. Bc4 Qh4+ 4. Kf1 Nc6 5. Nf3 Qd8 6. d4 Nf6 7. Nc3 d5 8. Nxd5 Nxd5 9. exd5 Nb4 10. Ne5 Be7 11. Nxf7 ) ( 1. e4 e5 2. f4 exf4 3. Bc4 Qh4+ 4. Kf1 Bc5 5. d4 Bd6 6. Nf3 Qh6 7. Nc3 Ne7 8. Ne2 f6 9. e5 fxe5 10. dxe5 Bc5 11. Nxf4 ) ( 1. e4 e5 2. f4 exf4 3. Bc4 Qh4+ 4. Kf1 d6 5. Nc3 Be6 6. Nd5 Kd8 7. d3 g5 8. g3 Qh6 9. h4 c6 10. Nc3 Nd7 11. Bxe6 ) ( 1. e4 e5 2. f4 exf4 3. Bc4 Qh4+ 4. Kf1 Nc6 5. Nf3 Qe7 6. Nc3 d6 7. d4 Nf6 8. Nd5 Qd8 9. Bxf4 Nxe4 10. Qd3 f5 11. Re1 ) ( 1. e4 e5 2. f4 exf4 3. Bc4 Qh4+ 4. Kf1 Nc6 5. Nf3 Qh5 6. d4 g5 7. Nc3 Bg7 8. Nd5 Kd8 9. Be2 Qg6 10. Nxg5 Qxg5 11. Bxf4 ) ( 1. e4 e5 

Now copy and paste the above into your preferred analysis engine!

I recommend Lichess study, as this will auto condense repeated lines.